In [1]:
import tensorflow as tf

# =====================
# LOAD BASELINE MODEL
# =====================
model = tf.keras.models.load_model(r"C:\Users\adity\Downloads/Code/Knowledge_Distillation/baseline_mobilenetv3_final.keras")

# =====================
# TFLITE CONVERTER
# =====================
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Optimisasi default
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# INT16 quantization (weights only)
converter.target_spec.supported_types = [tf.int16]

# Penting: pastikan input-output tetap float
converter.inference_input_type = tf.float32
converter.inference_output_type = tf.float32

# =====================
# CONVERT
# =====================
tflite_int16 = converter.convert()

# =====================
# SAVE MODEL
# =====================
with open(r"C:\Users\adity\Downloads/Code/Quantization/testdataluar_int16.tflite", "wb") as f:
    f.write(tflite_int16)

print("✅ INT16 TFLite model saved")


C:\Users\adity\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 22 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


INFO:tensorflow:Assets written to: C:\Users\adity\AppData\Local\Temp\tmpgqfznx8i\assets


INFO:tensorflow:Assets written to: C:\Users\adity\AppData\Local\Temp\tmpgqfznx8i\assets


Saved artifact at 'C:\Users\adity\AppData\Local\Temp\tmpgqfznx8i'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  2364815336144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2364815336720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2364815337296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2364815337680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2364815336912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2364815335952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2364815339600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2364815338448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2364815338832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2364815338256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  236

In [3]:
interpreter = tf.lite.Interpreter(
    model_path=r"C:\Users\adity\Downloads/Code/Quantization/testdataluar_int16.tflite",
    experimental_delegates=[]
)

interpreter.allocate_tensors()

input_details  = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input :", input_details)
print("Output:", output_details)


Input : [{'name': 'serving_default_input_layer_1:0', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([ -1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output: [{'name': 'StatefulPartitionedCall_1:0', 'index': 218, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([-1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


C:\Users\adity\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [6]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input

# =====================
# CONFIG
# =====================
IMG_SIZE = 224
BATCH_SIZE = 16
TEST_DIR = r"C:\Users\adity\Downloads/Tomato Leaf Disease Dataset/TomatoDataset"


# =====================
# TEST GENERATOR (SAMA DENGAN TRAINING)
# =====================
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_gen = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

print("✅ test_gen ready")
print("Total test samples:", test_gen.samples)


Found 1609 images belonging to 10 classes.
✅ test_gen ready
Total test samples: 1609


In [7]:
import time
import numpy as np

correct = 0
total = 0
times = []

for i in range(test_gen.samples):
    x_batch, y_batch = next(test_gen)

    x = x_batch[0:1].astype(np.float32)
    y = y_batch[0]

    start = time.time()
    interpreter.set_tensor(input_details[0]["index"], x)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]["index"])
    end = time.time()

    pred = output.argmax(axis=1)[0]
    true = y.argmax()

    correct += int(pred == true)
    total += 1
    times.append(end - start)

accuracy = correct / total
avg_time = sum(times) / len(times)
fps = 1 / avg_time

print("\n✅ INT16 TFLite RESULTS")
print(f"Accuracy            : {accuracy*100:.2f}%")
print(f"Avg inference/image : {avg_time:.6f} sec")
print(f"FPS                 : {fps:.2f}")



✅ INT16 TFLite RESULTS
Accuracy            : 46.74%
Avg inference/image : 0.003685 sec
FPS                 : 271.37
